# Formulación del Problema de Asignación de Recursos

Considere tres posiciones a llenar en Pantaleón: Tester, Java-Developer y Architect.

Considere tres posibles candidatos, en los que todos conocen las tres disciplinas: Carlos, Joe, y Monika.

## Datos

Se realizó una evaluación a cada candidato para medir la habilidad para realizar cada uno de estos trabajos. Las notas obtenidas se resumen a continuación:

![Resource Allocation Problem Data Image](https://github.com/Gurobi/modeling-examples/blob/master/milp_tutorial/util/rap_data.png?raw=1)


**Asunción**: Solamente un recurso puede trabajar en una posición, y solamente una posición puede asignarse a un recurso.

## Descripción del Problema

Determine una asignación que asegure que cada posición sea llenada, y que cada recurso se asigne a lo más a una posición para *maximizar* la puntuación total de las asignaciones.

## Variables de Decisión

Se define la variable de decisión $x_{r,\; j} = 1$ que representa que el recurso $r$ es asignado a la posición $j$, y 0 en otro caso, para todos los recursos $r=1,2,3$ y posiciones $𝑗=1,2,3$.

## Restricciones

### Restricciones de Posición

Para cada posición $𝑗=1,2,3,$ exactamente un recurso de $r=1,2,3$ debe ser asignado.

Restricción (Tester=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

Restricción (Java-Developer=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

Restricción (Architect=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

### Restricciones de Recursos

Para cada recurso $r=1,2,3$, a lo más una posición $𝑗=1,2,3,$ puede ser asignada.

Restricción (Carlos=1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Restricción (Joe=2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Restricción (Monika=3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

## Función Objetivo

Se formula una función objetivo para *maximizar* el total de puntuación de las asignaciones mientras se satisfacen las restricciones de posiciones y recursos.

$$
\max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




Instalación de paquetes para Optimización

In [15]:
%pip install gurobipy
%pip install names
%pip install numpy

In [16]:
# Gurobi
from gurobipy import *

## Datos
En una lista 'R' se colectan los nombres de los recursos: Carlos, Joe y Monika.

En una lista 'J' se colectan los nombres de las posiciones disponibles: teste, java-developer y architect.


**Notación Matemática**

$r \in R$ significa que un recurso con índice $r$ está en el conjunto (lista) 'R'.

$j \in J$ significa que una posición con índice $j$ está en el conjunto (lista) 'J'.

In [17]:
# conjuntos de recursos y posiciones
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

La función "multidict" describe las puntuaciones para cada trabajo asociadas con cada combinación de un recurso y una posición (tabla de notas obtenidas).


**Notación Matemática**

Sea $ms_{r,\;j}$ la nota obtenida del recurso  $r \in R$  con respecto a la posición  $j \in J$.

Sea $C_{r,\;j}$ el costo de asignar el recurso  $r \in R$  a la posición  $j \in J$.

Let $B$ el presupuesto disponible.

In [18]:
# tabla de puntuaciones y costos por recurso y posición
combinations, ms, C = multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# Presupuesto disponible
#B = 6
B=5

La siguiente función genera un objeto modelo vacío "m" y toma como argumento el nombre 'RAP' (Resource Assignment Problem)

In [19]:
# Se declara e inicializa el modelo
m = Model('RAP')

## Variables de Decisión

La variable de decisión $x_{r,\; j} = 1$ representa que el recurso $r$ es asignado a la posición $j$, y 0 en otro caso, para $r=1,2,3$ Y $𝑗=1,2,3$.

El método “addVars()” define las variables de decisión del objeto modelo "m".

**Notación Matemática**

Sea $x_{r,\; j} = 1$ si el recurso $r \in R$  se asigna a la posición $j \in J$, y cero en otro caso.

Sea $g_{j} = 1$ si la posición $j \in J$ no puede ser llenada, y cero en otro caso. Esta variable es una variable "gap" que indica que una posición no puede ser llenada.


In [20]:
# Crear las variables de decisión para el modelo RAP
#x = m.addVars(combinations, name="assign")
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# Crear las variables "gap" del modelo RAP
g = m.addVars(J, name="gap")

## Resticciones de posición

Para cada posición $𝑗=1,2,3$, exactamente un recurso de $r=1,2,3$ debe ser asignado.

Restricción (Tester=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

Restricción (Java-Developer=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

Restricción (Architect=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

El método “addConstrs()” define las restricciones del objeto modelo "m".


**Notación matemática**

Para cada posición $j \in J$, exactamente un recurso puede ser asignado:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1
$$



In [21]:
# crear resticciones de posición
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), 'job')

## Restricciones de recursos

Para cada recurso $r=1,2,3$ a lo más una posición de $j=1,2,3$ puede asignarse.

Restricción (Carlos=1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Restricción (Joe=2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Restricción (Monika=3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

El método “addConstrs()” define las restricciones del objeto modelo "m".

**Notación Matemática**

Para cada recurso $r \in R$, a lo más una posición puede ser asignada:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1
$$

In [22]:
# crear restricciones de recursos
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

## Restricción de Presupuesto

El costo total de asignar recursos a posiciones debe ser menor o igual que el presupuesto disponible.

$$
\sum_{r \; \in \; R} \sum_{j \; \in \; J} C_{r, j}x_{r, j} \leq B
$$

In [23]:
budget = m.addConstr((x.prod(C) <= B), 'budget')

## Función Objetivo

La función objetivo se plantea para maximizar el punteo total de todas las asignaciones de recursos a posiciones.

$$
\max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

El método “setObjective()” define la función objetivo del objeto modelo "m".

**Notación matemática**

Nótese que:
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

Por lo tanto, la función objetivo puede ser expresada como sigue:

$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j} -BigM \sum_{j \in J} g_{j}
$$


In [24]:
# Penalización por no llenar una posición
BIGM =101

# El objetivo es maximizar el punteo total de las asignaciones de recursos a posiciones
m.setObjective(x.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)

In [25]:
# save model for inspection
m.write('RAP3.lp')
m.display()

Maximize
53.0 assign[Carlos,Tester] + 27.0 assign[Carlos,JavaDeveloper]
+ 13.0 assign[Carlos,Architect] + 80.0 assign[Joe,Tester]
+ 47.0 assign[Joe,JavaDeveloper] + 67.0 assign[Joe,Architect]
+ 53.0 assign[Monika,Tester] + 73.0 assign[Monika,JavaDeveloper]
+ 47.0 assign[Monika,Architect] + -101.0 gap[Tester] + -101.0 gap[JavaDeveloper] +
-101.0 gap[Architect]
Subject To
job[Tester]: assign[Carlos,Tester] + assign[Joe,Tester] + assign[Monika,Tester] +
 gap[Tester] = 1
job[JavaDeveloper]: assign[Carlos,JavaDeveloper] + assign[Joe,JavaDeveloper] +
 assign[Monika,JavaDeveloper] + gap[JavaDeveloper] = 1
job[Architect]: assign[Carlos,Architect] + assign[Joe,Architect] +
 assign[Monika,Architect] + gap[Architect] = 1
resource[Carlos]: assign[Carlos,Tester] + assign[Carlos,JavaDeveloper] +
 assign[Carlos,Architect] <= 1
resource[Joe]: assign[Joe,Tester] + assign[Joe,JavaDeveloper] + assign[Joe,Architect]
 <= 1
resource[Monika]: assign[Monika,Tester] + assign[Monika,JavaDeveloper] +
 assign[Mon

In [26]:
# correr el motor de optimización
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0

In [27]:
# despliegue de las variables de decisión que lograron un valor óptimo
for v in m.getVars():
	if (abs(v.x) > 1e-6):
		print(v.varName, v.x)

# desplegar el punteo total alcanzado
print('Valor óptimo de la función objetivo:', m.objVal)

# Calculo del punteo total basado en las variables de asignación
total_matching_score = 0
for [r, j] in combinations:
    if (abs(x[r, j].x) > 1e-6):
        total_matching_score = total_matching_score + ms[r, j]*x[r, j].x

print('Punteo total logrado: ', total_matching_score)


assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
gap[Architect] 1.0
Valor óptimo de la función objetivo: 52.0
Punteo total logrado:  153.0
